# Trabalho de Engenharia de Dados II

## Membros
* Clairton Carneiro Luz
* Lairton Mendes
* Paulo Roberto Mesquita da Silva

## Instalando dependências

In [ ]:
!pip install -r requirements.txt

## Baixando dataset

In [8]:

!mkdir -p datasets

import gdown

url = 'https://drive.google.com/uc?id=1stubK6570vA8beNB2lRwWyxYO7lL4GDh'
output = 'datasets/onlinefoods.csv'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1stubK6570vA8beNB2lRwWyxYO7lL4GDh
To: /Users/clairtonluz/projects/personal/MBA-em-ciencia-de-dados/Engenharia de Dados II/trabalho/datasets/onlinefoods.csv
100%|██████████| 35.8k/35.8k [00:00<00:00, 720kB/s]


'datasets/onlinefoods.csv'

## Carregando dataset

In [11]:
# Carregando = 'datasets/onlinefoods.csv' usando spark que está rodando localmente using componse.yml

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder \
    .appName("OnlineFoods") \
    .getOrCreate()

sc = spark.sparkContext

df=spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .csv('./datasets/onlinefoods.csv')

df.show(5)


+---+------+--------------+----------+--------------+--------------------------+-----------+--------+---------+--------+------+---------+---------+
|Age|Gender|Marital Status|Occupation|Monthly Income|Educational Qualifications|Family size|latitude|longitude|Pin code|Output| Feedback|Recommend|
+---+------+--------------+----------+--------------+--------------------------+-----------+--------+---------+--------+------+---------+---------+
| 20|Female|        Single|   Student|     No Income|             Post Graduate|          4| 12.9766|  77.5993|  560001|   Yes| Positive|      Yes|
| 24|Female|        Single|   Student|Below Rs.10000|                  Graduate|          3|  12.977|  77.5773|  560009|   Yes| Positive|      Yes|
| 22|  Male|        Single|   Student|Below Rs.10000|             Post Graduate|          3| 12.9551|  77.6593|  560017|   Yes|Negative |      Yes|
| 22|Female|        Single|   Student|     No Income|                  Graduate|          6| 12.9473|  77.5616| 

In [12]:
# mostre os valores distintos da coluna 'Monthly Income'
df.select('Monthly Income').distinct().show()

+---------------+
| Monthly Income|
+---------------+
|      No Income|
| 10001 to 25000|
| 25001 to 50000|
|More than 50000|
| Below Rs.10000|
+---------------+



## Transformando dado

In [13]:
# crie uma nova columa chamada 'MonthlyIncomme' que vai seguir o seguinte padrão:
# - se 'No Income', 'MonthlyIncome' = '0'
# - se 'Below Rs.10000', 'MonthlyIncome' = '5000'
# - se '10000 to 20000', 'MonthlyIncome' = '15000'
# - se '25001 to 50000', 'MonthlyIncome' = '37500'
# - se 'More than 50000', 'MonthlyIncome' = '50000'
from pyspark.sql.functions import when

df = df.withColumn('MonthlyIncome', when(df['Monthly Income'] == 'No Income', 0)
                    .when(df['Monthly Income'] == 'Below Rs.10000', 5000)
                    .when(df['Monthly Income'] == '10000 to 20000', 15000)
                    .when(df['Monthly Income'] == '25001 to 50000', 37500)
                    .when(df['Monthly Income'] == 'More than 50000', 50000)
                    .otherwise(0))

df.show(5)


+---+------+--------------+----------+--------------+--------------------------+-----------+--------+---------+--------+------+---------+---------+-------------+
|Age|Gender|Marital Status|Occupation|Monthly Income|Educational Qualifications|Family size|latitude|longitude|Pin code|Output| Feedback|Recommend|MonthlyIncome|
+---+------+--------------+----------+--------------+--------------------------+-----------+--------+---------+--------+------+---------+---------+-------------+
| 20|Female|        Single|   Student|     No Income|             Post Graduate|          4| 12.9766|  77.5993|  560001|   Yes| Positive|      Yes|            0|
| 24|Female|        Single|   Student|Below Rs.10000|                  Graduate|          3|  12.977|  77.5773|  560009|   Yes| Positive|      Yes|         5000|
| 22|  Male|        Single|   Student|Below Rs.10000|             Post Graduate|          3| 12.9551|  77.6593|  560017|   Yes|Negative |      Yes|         5000|
| 22|Female|        Single| 

In [18]:
# Exibe quem gasta mais em média, solteiros ou casados, ordenando do maior para o menor
df.groupBy('Marital Status') \
    .mean('MonthlyIncome') \
    .sort('avg(MonthlyIncome)', ascending=False) \
    .show()


+-----------------+------------------+
|   Marital Status|avg(MonthlyIncome)|
+-----------------+------------------+
|          Married| 32592.59259259259|
|Prefer not to say|           26875.0|
|           Single| 7350.746268656716|
+-----------------+------------------+



In [19]:
# Quem gasta mais em média, por ocupação, ordenando do maior para o menor
df.groupBy('Occupation') \
    .mean('MonthlyIncome') \
    .sort('avg(MonthlyIncome)', ascending=False) \
    .show()

+--------------+------------------+
|    Occupation|avg(MonthlyIncome)|
+--------------+------------------+
|Self Employeed| 32870.37037037037|
|      Employee|31694.915254237287|
|       Student|1437.1980676328503|
|    House wife|               0.0|
+--------------+------------------+



In [20]:
# quem gasta mais por tamanho de família, ordenando do maior para o menor
df.groupBy('Family size') \
    .mean('MonthlyIncome') \
    .sort('avg(MonthlyIncome)', ascending=False) \
    .show()

+-----------+------------------+
|Family size|avg(MonthlyIncome)|
+-----------+------------------+
|          6|           27500.0|
|          3| 15641.02564102564|
|          1|           14375.0|
|          4|13293.650793650793|
|          2|13094.059405940594|
|          5|12592.592592592593|
+-----------+------------------+

